In [1]:
import os
import sys
import pandas as pd
import numpy as np
import folium
import io
from PIL import Image
from IPython.display import display


from helpers import (
    peform_mds_division,
)

path = os.path.abspath("../spatial-awareness/geocoding/")
sys.path.append(path)
from geocoding import GeoCoding

%load_ext autoreload
%autoreload 2

In [3]:
cities = pd.read_pickle('cities_opt.pkl')
cities_list = list(set(cities.a_name.to_list()))

gc = GeoCoding()

coods = []
coods_dic = {}
for each in cities_list:
    lat, lng, _, _ , _ = gc.get_lat_lng(each)
    coods.append([lng, lat])
    coods_dic[each] = [lat, lng]
    
coods = np.array(coods)

dis_disparity_df = pd.DataFrame(0.0,index=cities_list, columns=cities_list)

for i, each in dis_disparity_df.iterrows():
    for c in cities_list:
        if i == c:
            continue
        d = cities.loc[(cities.a_name == i) & (cities.b_name == c)]
        each[c] = d['distance'].to_list()[0]

vals = peform_mds_division(
    disparity_df=dis_disparity_df,
    coods_dic=coods_dic,
    metric=True,
    asymmetric=False,
    similarity_measure_used=False
)
df_1 = pd.DataFrame(vals)
df_1.rename(columns={
    'pred_lat': 'lat_dis',
    'pred_lng': 'lng_dis',
    'err': 'err_dis',
}, inplace=True)

pred_dis_disparity_df = pd.DataFrame(0.0,index=cities_list, columns=cities_list)

for i, each in pred_dis_disparity_df.iterrows():
    for c in cities_list:
        if i == c:
            continue
        d = cities.loc[(cities.a_name == i) & (cities.b_name == c)]
        val = d['predicted_dis'].to_list()[0]
        if pd.isnull(val):
            val = cities['predicted_dis'].mean()
        each[c] = val

vals_pred = peform_mds_division(
    disparity_df=pred_dis_disparity_df,
    coods_dic=coods_dic,
    metric=True,
    asymmetric=True,
    similarity_measure_used=False
)
df_2 = pd.DataFrame(vals_pred)
df_2.rename(columns={
    'pred_lat': 'lat_pred_dis',
    'pred_lng': 'lng_pred_dis',
    'err': 'err_pred_dis',
}, inplace=True)

df = pd.merge(df_1, df_2, on=['name', 'lat', 'lng'])

def get_all_coods(name, df=df):
    t = df.loc[df.name==name]
    return {
        'original': [t['lat'].item(), t['lng'].item()],
        'dis': [t['lat_dis'].item(), t['lng_dis'].item()],
        'pred_dis': [t['lat_pred_dis'].item(), t['lng_pred_dis'].item()],
    }


In [4]:

def add_markers_to_map(map, name, fill_opacity_dis=False, fill_opacity_pred=False, fill_opacity_orig=False):
    coods = get_all_coods(name)
    if fill_opacity_orig:
        folium.Marker(
            coods['original'],
            icon=folium.Icon(color='green', icon='circle', prefix='fa'),
        ).add_to(map)
    else:
        folium.Marker(
            coods['original'],
            icon=folium.Icon(color='green', icon='circle', prefix='fa'),
            opacity=0.7
        ).add_to(map)

    if fill_opacity_dis:
        folium.Marker(
            coods['dis'],
            icon=folium.Icon(color='blue', icon='circle', prefix='fa'),
            opacity=0.7,
        ).add_to(map)
    else:
        folium.Marker(
            coods['dis'],
            icon=folium.Icon(color='blue', icon='circle', prefix='fa'),
        ).add_to(map)

    if fill_opacity_pred:
        folium.Marker(
            coods['pred_dis'],
            icon=folium.Icon(color='red', icon='circle', prefix='fa'),
        ).add_to(map)
    else:
        folium.Marker(
            coods['pred_dis'],
            icon=folium.Icon(color='red', icon='circle', prefix='fa'),
        ).add_to(map)

def generate_map(name, save_name, fill_opacity_dis=False, fill_opacity_pred=False, fill_opacity_orig=False):
    zoom = 6
    m = folium.Map(
        location=get_all_coods(name)['original'],
        # tiles="Stamen Toner",
        zoom_start=zoom,
    )

    add_markers_to_map(m, name, fill_opacity_dis, fill_opacity_pred, fill_opacity_orig)
    display(m)
    img_data = m._to_png(10)
    img = Image.open(io.BytesIO(img_data))
    img.show()
    img.save(f'map_images_opt/coods_preds-{save_name}.png')

In [5]:
# Albany, New York
name = 'Albany, New York'
generate_map(name, 'albany')

In [6]:
# Havre, Montana
name = 'Havre, Montana'
generate_map(name, 'havre')

In [7]:
# Dallas, Texas
name = 'Dallas, Texas'
generate_map(name, 'dallas', fill_opacity_dis=True)

In [8]:
# Indianapolis, Indiana
name = 'Indianapolis, Indiana'
generate_map(name, 'indianapolis', fill_opacity_pred=True)